In [4]:
import random

import pandas as pd
import numpy as np
from scipy import stats

# from oauth2client.service_account import ServiceAccountCredentials 
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials 

In [5]:
CREDENTIALS_PATH_GOOGLE = '../google-credentials.json'
SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']
SPREADSHEET = '1b75J-QTGrujSgF9r0_JPOKkcXAwzFVwpETOAyVBw8ak'

In [6]:
# Load service account credentials.
__credentials = Credentials.from_service_account_file(CREDENTIALS_PATH_GOOGLE, scopes=SCOPES)

# Creates Google Sheets API (v4/latest) service.
service = build('sheets', 'v4', credentials=__credentials)

FileNotFoundError: [Errno 2] No such file or directory: '../google-credentials.json'

In [38]:
# Gets values from Ach! Musik: Notations sheet.
values = service.spreadsheets().values().get(spreadsheetId=SPREADSHEET, range='Notations').execute()['values']
headers = values.pop(0)

In [39]:
# Format data as pd.DataFrame
data = pd.DataFrame(values, columns=headers)
data

,genre,sub_genre,artist,album,song,Qu,Gr,Vi,Ro,Sa,Gl,Rx,Cl,Lu
0,Rock,Alternative,4 Non Blondes,"Bigger, Better, Faster, More!",What's Up,"8,75","8,5",,"8,7",None,None,None,None,None
1,Pop,Synth,A-Ha,Hunting High and Low,Take on Me,8,,,9,None,None,None,None,None
2,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",8,"8,5",5,8,,,,,5
3,Pop,Disco,Abba,Abba,Mamma Mia,"8,5","8,75","6,5","8,3",,6,None,None,None
4,Pop,Disco,Abba,Abba,SOS,9,9,7,"8,4",,"6,25",None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",2001,The Next Episode,,"8,2",None,None,None,None,None,None,None
2052,Hip-Hop,Rap,"Dr. Dre, Eminem",2001,Forgot About Dre,,7,None,None,None,None,None,None,None
2053,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",The Chronic,"Nuthin' But A ""G"" Thang",,"7,75",None,None,None,None,None,None,None
2054,Hip-Hop,Rap,2Pac,All Eyez On Me,Ambitionz Az A Ridah,,"7,75",None,None,None,None,None,None,None


In [40]:
# Saving as csv for later use
data.to_csv("../data/achmusik.csv", index=False, decimal=",")

In [7]:
data = pd.read_csv("../data/achmusik.csv")
data

,genre,sub_genre,artist,album,song,Qu,Gr,Vi,Ro,Sa,Gl,Rx,Cl,Lu
0,Rock,Alternative,4 Non Blondes,"Bigger, Better, Faster, More!",What's Up,"8,75","8,5",NaN,"8,7",NaN,NaN,NaN,NaN,NaN
1,Pop,Synth,A-Ha,Hunting High and Low,Take on Me,8,NaN,NaN,9,NaN,NaN,NaN,NaN,NaN
2,Pop,Disco,Abba,Abba,"I do, I do, I do, I do, I do",8,"8,5",5,8,NaN,NaN,NaN,NaN,5
3,Pop,Disco,Abba,Abba,Mamma Mia,"8,5","8,75","6,5","8,3",NaN,6,NaN,NaN,NaN
4,Pop,Disco,Abba,Abba,SOS,9,9,7,"8,4",NaN,"6,25",NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2051,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",2001,The Next Episode,NaN,"8,2",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2052,Hip-Hop,Rap,"Dr. Dre, Eminem",2001,Forgot About Dre,NaN,7,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2053,Hip-Hop,Rap,"Dr. Dre, Snoop Dogg",The Chronic,"Nuthin' But A ""G"" Thang",NaN,"7,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN
2054,Hip-Hop,Rap,2Pac,All Eyez On Me,Ambitionz Az A Ridah,NaN,"7,75",NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Getting the decimals right -- commas to points and no more Nones
data = data.set_index(["genre", "sub_genre", "artist", "album", "song"])
data.fillna(value="", inplace=True)

for i in range(data.columns.size):
    data[data.columns[i]] = data[data.columns[i]].str.replace(",", ".")
    data[data.columns[i]] = pd.to_numeric(data[data.columns[i]], errors='coerce')

In [49]:
kept_people = ["Qu", "Gr", "Vi", "Ro"]
default_grade = 5

# Keeping only present people at the hypothetical party!
data = data.filter(kept_people)

# Hard to do this shit inplace -- if no grades at all, give it a chance to play with default grade
data = data.dropna(how="all").append(data[data.isnull().all(axis=1)].fillna(default_grade))
data

Qu  \
genre sub_genre   artist            album                         song                                 
Rock  Alternative 4 Non Blondes     Bigger, Better, Faster, More! What's Up                     8.75   
Pop   Synth       A-Ha              Hunting High and Low          Take on Me                    8.00   
      Disco       Abba              Abba                          I do, I do, I do, I do, I do  8.00   
                                                                  Mamma Mia                     8.50   
                                                                  SOS                           9.00   
...                                                                                              ...   
Metal Symphonique Within Temptation The Unforgiving               Murder                        5.00   
                                                                  Shot in the Dark              5.00   
                                                                  Sinead                        5.00   
                                                                  Stairway to the Skies         5.00   
                                                                  Where is the Edge             5.00   

                                                                                                  Gr  \
genre sub_genre   artist            album                         song                                 
Rock  Alternative 4 Non Blondes     Bigger, Better, Faster, More! What's Up                     8.50   
Pop   Synth       A-Ha              Hunting High and Low          Take on Me                     NaN   
      Disco       Abba              Abba                          I do, I do, I do, I do, I do  8.50   
                                                                  Mamma Mia                     8.75   
                                                                  SOS                           9.00   
...                                                                                              ...   
Metal Symphonique Within Temptation The Unforgiving               Murder                        5.00   
                                                                  Shot in the Dark              5.00   
                                                                  Sinead                        5.00   
                                                                  Stairway to the Skies         5.00   
                                                                  Where is the Edge             5.00   

                                                                                                 Vi  \
genre sub_genre   artist            album                         song                                
Rock  Alternative 4 Non Blondes     Bigger, Better, Faster, More! What's Up                     NaN   
Pop   Synth       A-Ha              Hunting High and Low          Take on Me                    NaN   
      Disco       Abba              Abba                          I do, I do, I do, I do, I do  5.0   
                                                                  Mamma Mia                     6.5   
                                                                  SOS                           7.0   
...                                                                                             ...   
Metal Symphonique Within Temptation The Unforgiving               Murder                        5.0   
                                                                  Shot in the Dark              5.0   
                                                                  Sinead                        5.0   
                                                                  Stairway to the Skies         5.0   
                                                                  Where is the Edge             5.0   

                                                    

## Score voting

In [69]:
COUNT_FACTOR = .3
COUNT_INHIB = len(kept_people) // 2
MIN_SCORE = 6
PLAYLIST_SIZE = 200
ELIMINATING_GRADE = 4.9

# To avoid having to hard code the amount of columns for cases where the next cell is re-ran, we initialize columns
data["mean"] = 0
data["count"] = 0
data["score"] = 0
data["rank"] = 0

In [70]:
# Mean of all notes for each track
data["mean"] = data[data.columns[:-4]].mean(axis=1)

# Amount of notes for each track
data["count"] = data.count(axis=1) - 4

# Helping songs graded by more people in the group
data["score"] = data["mean"] + (COUNT_FACTOR * (data["count"] - COUNT_INHIB))

# Truncating to keep only the acceptable songs
data = data[data["score"] > MIN_SCORE]

data = data.sort_values("score", ascending=False)
data["rank"] = data["score"].rank(method="min")
data

Qu  \
genre   sub_genre   artist              album                   song                                  
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                   9.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            10.00   
                    Dire Straits        Dire Straits            Sultans Of Swing              10.00   
        Glam        David Bowie         Hunky Dory              Life on Mars                   9.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps   9.75   
...                                                                                             ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             6.50   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace          6.25   
                                                                Patinoire                      5.25   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life                6.25   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                     5.00   

                                                                                                Gr  \
genre   sub_genre   artist              album                   song                                 
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                  8.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            9.25   
                    Dire Straits        Dire Straits            Sultans Of Swing              9.25   
        Glam        David Bowie         Hunky Dory              Life on Mars                  9.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps  9.50   
...                                                                                            ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             NaN   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace          NaN   
                                                                Patinoire                      NaN   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life                NaN   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                    7.25   

                                                                                                Vi  \
genre   sub_genre   artist              album                   song                                 
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                  9.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            9.50   
                    Dire Straits        Dire Straits            Sultans Of Swing              9.50   
        Glam        David Bowie         Hunky Dory              Life on Mars                  8.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps   NaN   
...                                                                                            ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             NaN   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace         6.00   
                                                                Patinoire                     7.00   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life               6.00   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                     NaN   

                                                                                               Ro  \
ge

In [71]:
# Removing tracks with at least one grade under the minimum required
data = data[data[data.columns[:-4]].min(axis=1) > ELIMINATING_GRADE]
data

Qu  \
genre   sub_genre   artist              album                   song                                  
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                   9.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            10.00   
                    Dire Straits        Dire Straits            Sultans Of Swing              10.00   
        Glam        David Bowie         Hunky Dory              Life on Mars                   9.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps   9.75   
...                                                                                             ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             6.50   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace          6.25   
                                                                Patinoire                      5.25   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life                6.25   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                     5.00   

                                                                                                Gr  \
genre   sub_genre   artist              album                   song                                 
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                  8.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            9.25   
                    Dire Straits        Dire Straits            Sultans Of Swing              9.25   
        Glam        David Bowie         Hunky Dory              Life on Mars                  9.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps  9.50   
...                                                                                            ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             NaN   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace          NaN   
                                                                Patinoire                      NaN   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life                NaN   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                    7.25   

                                                                                                Vi  \
genre   sub_genre   artist              album                   song                                 
Rock    Psychedelic David Bowie         David Bowie             Space Oddity                  9.75   
        NaN         The Beatles         Abbey Road              Here Comes The Sun            9.50   
                    Dire Straits        Dire Straits            Sultans Of Swing              9.50   
        Glam        David Bowie         Hunky Dory              Life on Mars                  8.50   
        NaN         The Beatles         The Beatles             While My Guitar Gently Weeps   NaN   
...                                                                                            ...   
Metal   Symphonique Within Temptation   Mother Earth            Never Ending Story             NaN   
Hip-Hop NaN         Caballero           Laisse Nous Faire Vol.1 Discret Mais Efficace         6.00   
                                                                Patinoire                     7.00   
Jazz    Hip-Hop     Kenichiro Nishihara Jazzy Folklore          My Love My Life               6.00   
Metal   Heavy Metal Avenged Sevenfold   Avenged Sevenfold       Gunslinger                     NaN   

                                                                                               Ro  \
ge

In [315]:
if PLAYLIST_SIZE < 1:
    playlist = data.sample(frac=PLAYLIST_SIZE, weights="rank")
else:
    playlist = data.sample(n=PLAYLIST_SIZE, weights="rank")

playlist

Qu  \
genre   sub_genre   artist             album                     song                          
Dance   Electro     French 79          Olympic                   Vertigo Valley         8.25   
Rock    NaN         The Beatles        Revolver                  Eleanor Rigby          8.50   
        Progressive Pink Floyd         The Dark Side Of The Moon Breathe (In The Air)   9.50   
        Rockabilly  Elvis Presley      NaN                       Hound Dog              8.25   
        Progressive Supertramp         Breakfast in America      Oh Darling            10.00   
...                                                                                      ...   
Metal   Industrial  Marilyn Manson     The Pale Emperor          Killing Strangers      8.50   
Electro Disco       Todd Terje         It's Album Time           Strandbar              9.00   
Jazz    OST         Kristofer Maddigan Cuphead                   The King's Court       8.25   
Electro Pop         Parcels            Live Vol. 1               Enter                  8.75   
Jazz    OST         Kristofer Maddigan Cuphead                   Dramatic Fanatic       8.25   

                                                                                         Gr  \
genre   sub_genre   artist             album                     song                         
Dance   Electro     French 79          Olympic                   Vertigo Valley         NaN   
Rock    NaN         The Beatles        Revolver                  Eleanor Rigby         9.00   
        Progressive Pink Floyd         The Dark Side Of The Moon Breathe (In The Air)  8.75   
        Rockabilly  Elvis Presley      NaN                       Hound Dog             8.50   
        Progressive Supertramp         Breakfast in America      Oh Darling            8.75   
...                                                                                     ...   
Metal   Industrial  Marilyn Manson     The Pale Emperor          Killing Strangers      NaN   
Electro Disco       Todd Terje         It's Album Time           Strandbar              NaN   
Jazz    OST         Kristofer Maddigan Cuphead                   The King's Court       NaN   
Electro Pop         Parcels            Live Vol. 1               Enter                  NaN   
Jazz    OST         Kristofer Maddigan Cuphead                   Dramatic Fanatic       NaN   

                                                                                        Vi  \
genre   sub_genre   artist             album                     song                        
Dance   Electro     French 79          Olympic                   Vertigo Valley        7.0   
Rock    NaN         The Beatles        Revolver                  Eleanor Rigby         8.5   
        Progressive Pink Floyd         The Dark Side Of The Moon Breathe (In The Air)  NaN   
        Rockabilly  Elvis Presley      NaN                       Hound Dog             NaN   
        Progressive Supertramp         Breakfast in America      Oh Darling            8.0   
...                                                                                    ...   
Metal   Industrial  Marilyn Manson     The Pale Emperor          Killing Strangers     NaN   
Electro Disco       Todd Terje         It's Album Time           Strandbar             8.5   
Jazz    OST         Kristofer Maddigan Cuphead                   The King's Court      7.5   
Electro Pop         Parcels            Live Vol. 1               Enter                 9.0   
Jazz    OST         Kristofer Maddigan Cuphead                   Dramatic Fanatic      8.5   

                                                                                        Ro  \
genre   sub_genre   artist             album                     song                        
Dance   Electro     French 79          Olympic                   Vertigo Valley        NaN   
Rock    NaN         The Beatles        Revolver                  Eleanor Rigby         NaN   
      

## genre matrix and distances

In [316]:
DEFAULT_TRANSITION = "4,0"
DEFAULT_THRESHOLD = 7

transitions = pd.read_csv("../data/transitions.csv").fillna(DEFAULT_TRANSITION)
transitions.index = transitions["Unnamed: 0"]
transitions.drop("Unnamed: 0", axis=1, inplace=True)

# Getting the decimals right -- commas to points and no more Nones
for i in range(transitions.columns.size):
    transitions[transitions.columns[i]] = transitions[transitions.columns[i]].str.replace(",", ".")
    transitions[transitions.columns[i]] = pd.to_numeric(transitions[transitions.columns[i]], errors='coerce')

playlist.reset_index(inplace=True)

In [317]:
# This is so horribly un-optimized... May the Python Lords forgive me.

shuffled_playlist = playlist.iloc[0:1].drop(playlist.columns[5:], axis=1)
playlist.drop(0, inplace=True)
current_genre = shuffled_playlist.iloc[0]["genre"]
current_artist = shuffled_playlist.iloc[0]["artist"]

In [320]:
threshold = 8
chain = 0
chain_factor = .6
desperation_factor = .6
remove_indices = []

while playlist.size > 0:
    for row in playlist.iterrows():
        if (transitions[current_genre][row[1]["genre"]] + (chain * chain_factor) > threshold and row[1]["artist"] != current_artist) or threshold < 0:
            # Song accepted -- increment or reset chain
            if current_genre == row[1]["genre"]:
                chain += 1
            else:
                current_genre = row[1]["genre"]
                chain = 0

            # Add song to shuffled playlist and its index to a list for further removal
            shuffled_playlist = shuffled_playlist.append(playlist.loc[row[0]].drop(playlist.columns[5:]))
            remove_indices.append(row[0])
            threshold = DEFAULT_THRESHOLD
            
    # Removing songs that were added during the for loop
    if remove_indices:
        playlist.drop(remove_indices, inplace=True)
        remove_indices = []
    else:
        threshold -= desperation_factor

shuffled_playlist

,genre,sub_genre,artist,album,song
0,Dance,Electro,French 79,Olympic,Vertigo Valley
1,Rock,NaN,The Beatles,Revolver,Eleanor Rigby
2,Rock,Progressive,Pink Floyd,The Dark Side Of The Moon,Breathe (In The Air)
3,Rock,Rockabilly,Elvis Presley,NaN,Hound Dog
4,Rock,Progressive,Supertramp,Breakfast in America,Oh Darling
...,...,...,...,...,...
194,Metal,Progressive,Tool,Fear Inoculum,Pneuma
195,Metal,Industrial,Marilyn Manson,The Pale Emperor,Killing Strangers
196,Electro,Disco,Todd Terje,It's Album Time,Strandbar
152,Electro,Alternative,Gorillaz,Gorillaz,M1 A1


In [306]:
shuffled_playlist.to_csv("../test.csv", index=False)

In [249]:
transitions

,Acapella,Acoustic,Blues,Chanson,Classique,Country,Dance,Disco,Electro,Folk,Funk,Hip-Hop,Jazz,Metal,Musical,OST,Pop,R&B,Reggae,Rock
Unnamed: 0,,,,,,,,,,,,,,,,,,,,
Acapella,10,4.0,4,4.0,4.0,4.0,"3,25",4,4.0,4.0,4,4.0,4,1,4.0,4.0,4.0,4.0,4.0,4
Acoustic,4,10.0,4,4.0,4.0,4.0,4,4,4.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Blues,4,4.0,10,4.0,4.0,4.0,2,4,4.0,4.0,"3,5",4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Chanson,4,4.0,4,10.0,4.0,4.0,4,4,4.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Classique,4,4.0,4,4.0,10.0,4.0,4,4,4.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Country,4,4.0,4,4.0,4.0,10.0,4,4,4.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Dance,"3,25",4.0,2,4.0,4.0,4.0,10,4,4.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4
Disco,4,4.0,4,4.0,4.0,4.0,4,10,4.0,4.0,4,4.0,4,1,4.0,4.0,4.0,4.0,4.0,"5,75"
Electro,4,4.0,4,4.0,4.0,4.0,4,4,10.0,4.0,4,4.0,4,4,4.0,4.0,4.0,4.0,4.0,4


# Playground

In [42]:
data = data.reset_index()

In [150]:
COL = ["Qu", "Gr", "Vi"]
BY = "artist"
AMNT = 10

best = data[[BY, *COL]].dropna(how="any").groupby(BY).filter(lambda x: len(x) >= AMNT).groupby(BY).mean()[COL]
best[COL].mean(axis=1).sort_values(ascending=False).head(10)

artist
The Beatles         8.296474
The Doors           8.238636
Pink Floyd          8.069444
System Of A Down    7.986667
Ghost               7.800000
Daft Punk           7.732143
Gojira              7.666667
Gorillaz            7.601852
Muse                7.568182
Alice in Chains     7.282051
dtype: float64